# DSC | TMATM - Starter script -  LGBM Classifier

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

In [2]:
pd.set_option("display.max_rows",100)
pd.set_option("display.max_columns",100)

## Parameters

In [3]:
SEED = 42

## Read data

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("X_test.csv")

In [5]:
train.head()

,alert_id,true_alert,prties_prev_true_alerts_pct,prties_prev_true_alerts,prev_alerts,prties_prev_alerts,agrmnt_opposite_agrmnt_distinct_count_1d_sum_1m_avg_v1,prev_true_alerts_pct,agrmnt_opposite_agrmnt_distinct_count_1m_sum_6m_stddev_v1,prev_true_alerts,agrmnt_opposite_agrmnt_distinct_count_1m_sum_v1,agrmnt_opposite_agrmnt_distinct_count_1m_sum_6m_max_v1,all_prtys_first_contact_age_days_min_v1,all_prtys_first_contact_age_days_max_v1,agrmnt_opposite_agrmnt_distinct_count_1d_sum_5d_max_v1,agrmnt_count_credit_transfer_1d_sum_1m_avg_v1,all_prtys_all_accounts_count_v1,agrmnt_opposite_agrmnt_count_1d_sum_1m_stddev_v1,all_prtys_first_contact_age_days_avg_v1,agrmnt_opposite_agrmnt_distinct_count_1d_sum_1m_stddev_v1,all_prtys_special_credit_code_start_encode_ratio_v1_n,all_prtys_status_code_encode_ratio_v1_other,agrmnt_count_debit_1m_sum_6m_max_v1,agrmnt_count_debit_1m_sum_6m_sum_v1,agrmnt_product_id_v1,agrmnt_opposite_agrmnt_distinct_count_1m_sum_6m_sum_v1,agrmnt_count_debit_point_of_sale_1d_sum_5d_max_v1,all_prtys_employment_status_type_code_encode_ratio_v1_sem,agrmnt_opposite_agrmnt_count_1m_sum_6m_stddev_v1,agrmnt_count_credit_1d_sum_1m_stddev_v1,agrmnt_count_debit_regular_1m_sum_6m_avg_v1,agrmnt_street_address_id_v1,agrmnt_count_credit_transfer_1m_sum_6m_avg_v1,agrmnt_count_debit_transfer_1m_sum_6m_stddev_v1,all_prtys_status_classification_code_encode_ratio_v1_other,agrmnt_count_credit_transfer_1m_sum_6m_stddev_v1,agrmnt_count_debit_transfer_1m_sum_6m_min_v1,agrmnt_count_debit_regular_1d_sum_1m_stddev_v1,agrmnt_count_credit_1m_sum_6m_stddev_v1,agrmnt_count_credit_1m_sum_6m_avg_v1,agrmnt_count_credit_transfer_1m_sum_6m_max_v1,all_prtys_ledger_balance_debit_accounts_1m_avg_v1_sanitised,agrmnt_amount_credit_1m_sum_v1_sanitised,all_prtys_ledger_balance_debit_accounts_1m_max_v1_sanitised,all_prtys_ledger_balance_debit_accounts_1m_stddev_v1_sanitised,all_prtys_ledger_balance_debit_accounts_1m_min_v1_sanitised,agrmnt_amount_debit_1d_sum_1m_avg_v1_sanitised,agrmnt_amount_credit_transfer_1m_sum_6m_stddev_v1_sanitised,agrmnt_amount_debit_cash_1m_sum_6m_avg_v1_sanitised,agrmnt_amount_credit_transfer_1m_sum_v1_sanitised,agrmnt_amount_debit_1d_sum_1m_max_v1_sanitised,agrmnt_amount_debit_cash_1m_sum_6m_stddev_v1_sanitised,agrmnt_amount_credit_transfer_1d_sum_1m_min_v1_sanitised,agrmnt_amount_debit_regular_1m_sum_6m_sum_v1_sanitised,agrmnt_amount_credit_transfer_1d_sum_1m_stddev_v1_sanitised,agrmnt_amount_credit_1m_sum_6m_stddev_v1_sanitised,agrmnt_ledger_balance_1m_stddev_v1_sanitised,agrmnt_amount_credit_transfer_1d_sum_1m_max_v1_sanitised,agrmnt_amount_debit_1d_sum_5d_max_v1_sanitised,agrmnt_amount_debit_transfer_1m_sum_v1_sanitised,agrmnt_amount_credit_cash_1m_sum_6m_sum_v1_sanitised,agrmnt_amount_debit_transfer_1d_sum_1m_stddev_v1_sanitised,agrmnt_amount_credit_1m_sum_6m_min_v1_sanitised,agrmnt_amount_debit_transfer_1d_sum_1m_min_v1_sanitised,agrmnt_amount_credit_1d_sum_1m_avg_v1_sanitised,agrmnt_amount_debit_cash_1m_sum_6m_max_v1_sanitised,agrmnt_amount_debit_regular_1m_sum_6m_max_v1_sanitised,agrmnt_amount_credit_transfer_1d_sum_1m_avg_v1_sanitised,agrmnt_amount_debit_regular_1d_sum_5d_max_v1_sanitised,agrmnt_amount_debit_point_of_sale_1d_sum_1m_stddev_v1_sanitised,agrmnt_amount_credit_1d_sum_1m_max_v1_sanitised,agrmnt_amount_credit_transfer_1m_sum_6m_max_v1_sanitised,agrmnt_amount_debit_transfer_1d_sum_1m_max_v1_sanitised,agrmnt_amount_credit_charge_or_interest_1m_sum_6m_sum_v1_sanitised,agrmnt_ledger_balance_1m_max_v1_sanitised
0,24833,1,0.0,0,0,0,0.250000,0.0,0.000000,0,1.0,1,5417.0,5417.0,0.0,1.000000,4.0,0.500000,5417.0,0.500000,1.0,0.0,NaN,NaN,1173342,3,NaN,0.0,0.000000,0.000000,NaN,211332229.0,1.000000,NaN,0.0,0.000000,NaN,NaN,1.527525,2.333333,1.0,"(-67606.448, -19600.0]","(10674.762, 14557.404]","(-33696.221, -6551.256]","(3061.09, 6053.903]","(-35674.59, -15677.9]",NaN,"(5476.364, 8732.697]",NaN,"(0.009000000000000001, 919.6]",NaN,NaN,"(392.35, 1047.1]",NaN,NaN,"(5385.229, 7687.567]","(4217.64, 10567.478]","(350.551, 600.0]",Na

## Preprocessing

In [6]:
#Take the upper limit of the range for the sanitised columns and transform columns from object to float
for row in train.filter(like='sanitised').columns:
    train[row] = train[row].str.split(',').str[1].str[:-1].astype("float")

In [7]:
#Tranform object type columns to category
for object_cols in train.select_dtypes("object").columns:
        train[object_cols] = train[object_cols].astype("category")

In [8]:
#Select the columns to be used for modelling and create train and validation datasets
features = [x for x in list(train.columns) if x not in ['alert_id','true_alert']]

train, val = train_test_split(train, random_state=SEED, test_size=0.3)

## Modelling

In [9]:
model =  lgb.LGBMClassifier()

model.fit(train[features], train['true_alert'])

pred_train = model.predict(train[features])
pred_val = model.predict(val[features])

In [10]:
print(f'  Training Recall: {recall_score(train["true_alert"], pred_train):.2f}')
print(f'Validation Recall: {recall_score(val["true_alert"], pred_val):.2f}')

  Training Recall: 0.83
Validation Recall: 0.75


## Scoring test data

In [11]:
#Apply preprocessing to the test data 

In [12]:
#Take the upper limit of the range for the sanitised columns and transform columns from object to float
for row in test.filter(like='sanitised').columns:
    test[row] = test[row].str.split(',').str[1].str[:-1].astype("float")

In [13]:
#Tranform object type columns to category
for object_cols in test.select_dtypes("object").columns:
        test[object_cols] = test[object_cols].astype("category")

In [19]:
# Score the test dataset (please do not rename this column, as the column name needs to match the label ('true_alert') to prevent mlcomp errors)
test['true_alert'] = model.predict(test[features])

## Output submission 

In [20]:
#Prepare and write the submission file 
submission = test[['alert_id', 'true_alert']]
submission.to_csv('submission.csv', index=False)